In [1]:
import warnings;
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
#preprocessing ownership
df_ownership = pd.read_csv('Building_Ownership_Use.csv')
df_ownership['count_families']=df_ownership['count_families'].fillna(1.0)
df_ownership['has_secondary_use']=df_ownership['has_secondary_use'].fillna(0.0)
df_ownership.head()

,building_id,district_id,vdcmun_id,ward_id,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,a3380c4f75,7,701,70102,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0
1,a3380c4fd9,7,701,70102,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0
2,a3380c503e,7,701,70102,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0
3,a338a4e5f2,7,701,70103,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0
4,a338a4e653,7,701,70103,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0


In [3]:
#preprocessing ownership data
df_structure = pd.read_csv('Building_Structure.csv')
df_structure['count_floors_d']=df_structure['count_floors_pre_eq']-df_structure['count_floors_post_eq']
df_structure['height_ft_d']=df_structure['height_ft_pre_eq']-df_structure['height_ft_post_eq']
df_structure['height_ft_d']=df_structure['height_ft_d'].clip_lower(0)
df_structure['count_floors_d']=df_structure['count_floors_d'].clip_lower(0)
df_structure.drop(['count_floors_pre_eq','count_floors_post_eq','height_ft_pre_eq', 'height_ft_post_eq'],inplace=True,axis=1)
df_structure['position']=df_structure['position'].fillna('Not attached')
df_structure['plan_configuration']=df_structure['plan_configuration'].fillna('Rectangular')
df_structure.head()

,building_id,district_id,vdcmun_id,ward_id,age_building,plinth_area_sq_ft,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,count_floors_d,height_ft_d
0,a3380c4f75,7,701,70102,28,454,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Not applicable,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
1,a3380c4fd9,7,701,70102,32,324,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Not applicable,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
2,a3380c503e,7,701,70102,34,456,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
3,a338a4e5f2,7,701,70103,20,452,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
4,a338a4e653,7,701,70103,25,542,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Not applicable,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Rubble unclear,1,9


In [4]:
#combining structure and ownership details building wise
merged = pd.merge(df_ownership,df_structure, on=['building_id','district_id','vdcmun_id','ward_id'])
merged.head()

,building_id,district_id,vdcmun_id,ward_id,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_building,plinth_area_sq_ft,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,count_floors_d,height_ft_d
0,a3380c4f75,7,701,70102,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0,28,454,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Not applicable,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
1,a3380c4fd9,7,701,70102,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0,32,324,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Not applicable,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
2,a3380c503e,7,701,70102,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0,34,456,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
3,a338a4e5f2,7,701,70103,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0,20,452,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,TImber/Bamboo-Mud,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Repaired and used,0,0
4,a338a4e653,7,701,70103,Private,1.0,0.0,0,0,0,0,0,0,0,0,0,0,25,542,Moderate slope,Mud mortar-Stone/Brick,Bamboo/Timber-Light roof,Mud,Not applicable,Attached-1 side,Rectangular,0,1,0,0,0,0,1,1,0,0,1,Damaged-Rubble unclear,1,9


In [5]:
#reading training data
train= pd.read_csv('train.csv')
print(train.head())
train_final=pd.merge(train,merged,on=["building_id",'district_id','vdcmun_id'])
train.head()

       area_assesed  building_id damage_grade  district_id  \
0              Both  24385bfd2a2      Grade 4           24   
1              Both  405d1bbebbf      Grade 2           44   
2              Both  351d9bc71f6      Grade 1           36   
3  Building removed  2be3a971166      Grade 5           30   
4              Both  34c7d073ea6      Grade 3           36   

   has_geotechnical_risk  has_geotechnical_risk_fault_crack  \
0                    0.0                                  0   
1                    0.0                                  0   
2                    0.0                                  0   
3                    0.0                                  0   
4                    0.0                                  0   

   has_geotechnical_risk_flood  has_geotechnical_risk_land_settlement  \
0                            0                                      0   
1                            0                                      0   
2                            

,area_assesed,building_id,damage_grade,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,Both,24385bfd2a2,Grade 4,24,0.0,0,0,0,0,0,0,0,0.0,2489
1,Both,405d1bbebbf,Grade 2,44,0.0,0,0,0,0,0,0,0,1.0,4423
2,Both,351d9bc71f6,Grade 1,36,0.0,0,0,0,0,0,0,0,0.0,3650
3,Building removed,2be3a971166,Grade 5,30,0.0,0,0,0,0,0,0,0,0.0,3016
4,Both,34c7d073ea6,Grade 3,36,0.0,0,0,0,0,0,0,0,0.0,3627


In [6]:
#reading test data
test = pd.read_csv('test.csv')
test_final=pd.merge(test,merged,on=["building_id",'district_id','vdcmun_id'])

In [7]:
#filling null values
train_final['has_repair_started']=train_final['has_repair_started'].fillna(0)
test_final['has_repair_started']=test_final['has_repair_started'].fillna(0)


In [8]:
#preparing X and Y for training
train_final['damage_grade']=train_final['damage_grade'].str.replace('Grade ','').replace('5','0').astype(float)
Y=train_final['damage_grade']
X=train_final.drop(['damage_grade','building_id'],axis=1)
X[['district_id','vdcmun_id','ward_id']]=X[['district_id','vdcmun_id','ward_id']].astype(str)
categorical_features_indices = np.where(X.dtypes==object)[0]

In [9]:
#fitting model
model = CatBoostClassifier(iterations=30,depth=8, learning_rate=0.5,loss_function='MultiClassOneVsAll',classes_count=5)
model.fit(X,Y,cat_features=categorical_features_indices,plot=False,verbose=False)


Iteration with suspicious time -7.95e+03 sec ignored in overall statistics.


In [10]:
#fitting model on top seperately for class 2,3,4
train_new=train_final[train_final['damage_grade']>=2]
Y=train_new['damage_grade']-2
X=train_new.drop(['damage_grade','building_id'],axis=1)
X[['district_id','vdcmun_id','ward_id']]=X[['district_id','vdcmun_id','ward_id']].astype(str)
model_top = CatBoostClassifier(iterations=200,depth=8, learning_rate=0.6,loss_function='MultiClass',classes_count=3)
model_top.fit(X,Y,cat_features=categorical_features_indices,plot=False,verbose=True)

0:	learn: -0.9088221	total: 3.93s	remaining: 13m 1s
1:	learn: -0.8307476	total: 6.61s	remaining: 10m 54s
2:	learn: -0.7988087	total: 9.88s	remaining: 10m 48s
3:	learn: -0.7840204	total: 13s	remaining: 10m 34s
4:	learn: -0.7684394	total: 16s	remaining: 10m 22s
5:	learn: -0.7586908	total: 19.7s	remaining: 10m 36s
6:	learn: -0.7532438	total: 23s	remaining: 10m 34s
7:	learn: -0.7488234	total: 26.6s	remaining: 10m 37s
8:	learn: -0.7443575	total: 31s	remaining: 10m 57s
9:	learn: -0.7409807	total: 34.9s	remaining: 11m 3s
10:	learn: -0.7385631	total: 38.6s	remaining: 11m 3s
11:	learn: -0.7351743	total: 42.8s	remaining: 11m 11s
12:	learn: -0.7333944	total: 46.6s	remaining: 11m 10s
13:	learn: -0.7315410	total: 50.5s	remaining: 11m 11s
14:	learn: -0.7298882	total: 54.9s	remaining: 11m 17s
15:	learn: -0.7280348	total: 59.3s	remaining: 11m 21s
16:	learn: -0.7269570	total: 1m 3s	remaining: 11m 22s
17:	learn: -0.7252072	total: 1m 7s	remaining: 11m 25s
18:	learn: -0.7235667	total: 1m 11s	remaining: 11

152:	learn: -0.6728454	total: 11m 17s	remaining: 3m 28s
153:	learn: -0.6727827	total: 11m 21s	remaining: 3m 23s
154:	learn: -0.6724977	total: 11m 26s	remaining: 3m 19s
155:	learn: -0.6721430	total: 11m 30s	remaining: 3m 14s
156:	learn: -0.6719604	total: 11m 35s	remaining: 3m 10s
157:	learn: -0.6716850	total: 11m 39s	remaining: 3m 5s
158:	learn: -0.6713362	total: 11m 44s	remaining: 3m 1s
159:	learn: -0.6711200	total: 11m 49s	remaining: 2m 57s
160:	learn: -0.6708030	total: 11m 52s	remaining: 2m 52s
161:	learn: -0.6706272	total: 11m 57s	remaining: 2m 48s
162:	learn: -0.6704826	total: 12m 2s	remaining: 2m 43s
163:	learn: -0.6704024	total: 12m 7s	remaining: 2m 39s
164:	learn: -0.6701778	total: 12m 12s	remaining: 2m 35s
165:	learn: -0.6700338	total: 12m 17s	remaining: 2m 31s
166:	learn: -0.6699060	total: 12m 22s	remaining: 2m 26s
167:	learn: -0.6697092	total: 12m 27s	remaining: 2m 22s
168:	learn: -0.6694988	total: 12m 31s	remaining: 2m 17s
169:	learn: -0.6693630	total: 12m 36s	remaining: 2m 

In [11]:
#preparing X for test and predicting Y using trained model

X_test=test_final.drop(['building_id'],axis=1)
X_test[['district_id','vdcmun_id','ward_id']]=X_test[['district_id','vdcmun_id','ward_id']].astype(str)
test_final['damage_grade']=model.predict(X_test)
test_234= test_final[test_final['damage_grade']>=2]
X_test_234=test_234.drop(['building_id','damage_grade'],axis=1)
X_test_234[['district_id','vdcmun_id','ward_id']]=X_test_234[['district_id','vdcmun_id','ward_id']].astype(str)
test_234['damage_grade']=model_top.predict(X_test_234)+2
test_final.update(test_234)


In [12]:
#saving predicted results
sample=pd.read_csv('sample_submission.csv')
sample['damage_grade']=test_final['damage_grade']
sample['damage_grade']=sample['damage_grade'].astype(int).astype(str).str.replace('0','5')
sample['damage_grade']='Grade '+sample['damage_grade']
sample.to_csv('final.csv',index=False)

IOError: File sample_submission.csv does not exist